In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../src')


from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation
from sklearn.metrics import roc_auc_score
#from losses import FairODLoss
#import evaluate as e
#from model import training, plot_losses

In [3]:
# load adult
adult_df = pd.read_csv('../datasets/proc/crafted_adult.csv')

y_pv = adult_df[['OUTLIER','sex']]
X = adult_df.drop(columns=['OUTLIER', 'sex'])

X_train, X_test, y_pv_train, y_pv_test = train_test_split(X, y_pv, test_size=0.3, shuffle=True)

pv_test = y_pv_test['sex']
pv_train = y_pv_train['sex']
y_train = y_pv_train['OUTLIER']
y_test = y_pv_test['OUTLIER']

FairOD on adult dataset

In [4]:
model = OutlierDetector(alpha=0.1, gamma=0.8)
_, _ = model.fit(X_train, pv_train, batch_size=512, epochs=3, val_X=X_test, val_pv=pv_test)

Predict

In [5]:
# predict
X_pred = model.predict_scores(X_test)

# compute reconstruction error of predicted samples and save in dataframe
#outlier_score = evalu.compute_outlier_scores(X_test, X_pred)
outlier_score = X_pred
pred_df = pd.DataFrame({'Reconstruction_error':outlier_score, 'True_class': y_test})

Evaluate

In [6]:
# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, outlier_score, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, outlier_score, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, outlier_score, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(outlier_score, pv_test).numpy())

AP RATIO  2.405541295719654
PRECISION RATIO  2.188610265194772
FAIRNESS METRIC   0.9949820718401798
GROUP FIDELITY METRIC   1.0526297


In [7]:
print("AUC score:   ", roc_auc_score(y_test, X_pred))

AUC score:    0.5881847906226292


## SG-AE Method

In [8]:
from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation

In [9]:
model = SGOutlierDetector(epsilon=0.001, 
                          lambda_se=4.86, 
                          lambda_a=25.0,
                          a=20, 
                          alpha=0.01, 
                          gamma=0.25)


_, _ = model.fit(X_train, pv_train, batch_size=512, epochs=6, val_X=X_test, val_pv=pv_test)

In [10]:
# save predictions
X_pred = model.predict_scores(X_test).numpy()

# AUC score
print("AUC score:   ", roc_auc_score(y_test, X_pred))

AUC score:    0.8358152231155425


In [11]:
# FAIR OD
#AP RATIO  2.4286151127297075
#PRECISION RATIO  2.518340928996637
#FAIRNESS METRIC   0.69642915260362582
#GROUP FIDELITY METRIC   1.0554426


# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, X_pred, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, X_pred, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, X_pred, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(X_pred, pv_test).numpy())

AP RATIO  1.2862576301796047
PRECISION RATIO  1.205705705705706
FAIRNESS METRIC   0.42196830034667876
GROUP FIDELITY METRIC   0.0002470572
